# Data Collection and Cleaning

Data has been collected from both the EPA for Air Quality Index data and from the CDC for birth data relating to weight.  

## Imports

In [1]:
import pandas as pd
import numpy as np

from pathlib import Path

## EPA Air Quality System (AQS) Data

Data is in individual CSV files that were generated by the `AQS_query.py` script in the project repo and served by the Air Quality System (AQS) API.  This script requires an email and API key from https://aqs.epa.gov/aqsweb/documents/data_api.htmlhttps://aqs.epa.gov/aqsweb/documents/data_api.html  Due to the large amount of data that needed to be collected, this task was split between our project team members to collect various years of data.  Each state for each set of years generates a CSV file that need to be read in and concatenated so we can use for our project.

In [2]:
data_path = Path('data/')

# create a list of all the AQI csv files
aqs_dfs = [
    pd.read_csv(file) for file in data_path.glob('AQS_county*') if file.is_file()
]

# for file in data_path.glob('AQS_county*'):
#     if file.is_file():
#         print(file)
#         pd.read_csv(file)

# concatenate the dataframes
aqs = pd.concat(aqs_dfs)

### Important note: use validity indicator to drop unusable rows

After spending time with the data dictionary and other various resources from AirData on the aspects of the data we were collecting we decided to filter out certain values.  So here we only want data that has been indicated as valid.

In [3]:
aqs.shape

(271702, 57)

In [4]:
aqs = aqs[aqs['validity_indicator']=='Y']

In [5]:
aqs.shape

(224790, 57)

### Borrow EPA standard and only take measurements where `valid_day_count` > 75% of days in year

The EPA generally uses 75% as the cutoff for aggregation measures, so we will follow that here.
* [Source](https://aqs.epa.gov/aqsweb/documents/about_aqs_data.html#data-aggregation-summarization)

In [6]:
.75*365

273.75

In [7]:
# aqs = aqs[aqs['valid_day_count'] > .75*365]

In [8]:
aqs.shape

(224790, 57)

### Columns to Drop

As we went through the various stages of looking at all the data that was coming from the API collection, and considering what parameters and data we were focusing on for this project, we collected a list of the columns from the data to drop:

In [9]:
drop = ['state_code',
 'county_code',
 'site_number',
 'parameter_code',
 'poc',
 'latitude',
 'longitude',
 'city', 'cbsa_code', 'cbsa', 'date_of_last_change',
 'first_max_value',
  'first_max_datetime',
  'second_max_value',
  'second_max_datetime',
  'third_max_value',
  'third_max_datetime',
  'fourth_max_value',
  'fourth_max_datetime',
  'first_max_nonoverlap_value',
  'first_max_n_o_datetime',
  'second_max_nonoverlap_value',
  'second_max_n_o_datetime',
  'ninety_ninth_percentile',
  'ninety_eighth_percentile',
  'ninety_fifth_percentile',
  'ninetieth_percentile',
  'seventy_fifth_percentile',
  'fiftieth_percentile',
  'tenth_percentile',
  'local_site_name',
  'site_address',
 'datum',
 'units_of_measure',
 'sample_duration',
 'secondary_exceedance_count',
 'pollutant_standard',
 'metric_used',
 'method',
 'event_type',
 'observation_count',
 'observation_percent',
 'required_day_count',
 'exceptional_data_count',
 'null_observation_count']

In [10]:
aqs.drop(columns=drop, inplace=True)

In [11]:
aqs.head()

,parameter,sample_duration_code,year,validity_indicator,valid_day_count,primary_exceedance_count,certification_indicator,arithmetic_mean,standard_deviation,state,county,Unnamed: 0
0,Carbon monoxide,1,2006,Y,357,0.0,Certified,0.606517,0.325145,Indiana,Allen,NaN
1,Carbon monoxide,Z,2006,Y,357,0.0,Certified,0.610952,0.299519,Indiana,Allen,NaN
2,Ozone,1,2006,Y,180,0.0,Certified,0.051078,0.012712,Indiana,Allen,NaN
3,Ozone,1,2006,Y,174,0.0,Certified,0.052517,0.012942,Indiana,Allen,NaN
4,Ozone,W,2006,Y,180,0.0,Certified,0.045844,0.012638,Indiana,Allen,NaN


In [12]:
aqs.columns

Index(['parameter', 'sample_duration_code', 'year', 'validity_indicator',
       'valid_day_count', 'primary_exceedance_count',
       'certification_indicator', 'arithmetic_mean', 'standard_deviation',
       'state', 'county', 'Unnamed: 0'],
      dtype='object')

### Check County and State Information

Mainly looking for outliers or any text in the names that might be cause for further investigation, especially since we will be merging this data with the CDC data based on year, state, and county.

In [13]:
# commented out to prevent long output in the notebook
# aqs['county'].unique()

In [14]:
aqs['state'].unique()

array(['Indiana', 'Pennsylvania', 'Texas', 'Mississippi', 'Wyoming',
       'Wisconsin', 'Delaware', 'Iowa', 'Illinois', 'New Jersey',
       'Virgin Islands', 'Maryland', 'California', 'West Virginia',
       'Kentucky', 'Rhode Island', 'Arkansas', 'District Of Columbia',
       'Oklahoma', 'Massachusetts', 'Idaho', 'Kansas', 'Alabama',
       'Alaska', 'Arizona', 'Colorado', 'Connecticut', 'Florida',
       'Georgia', 'Hawaii', 'Louisiana', 'Maine', 'Michigan', 'Minnesota',
       'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
       'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio',
       'Oregon', 'South Carolina', 'South Dakota', 'Tennessee', 'Utah',
       'Vermont', 'Virginia', 'Washington', 'Puerto Rico'], dtype=object)

May want to drop locations such as:
* Puerto Rico

### Transform Data

The data came in as individual rows, but we need to transform/reshape the data so that the AQI pollutants are columns and the values match with the year/state/county they are reported in so that we can have a complete dataset for modeling.

EPA makes a recommendation to drop SO2 from data because it is more of a localized pollutant and can't be represented on a country or metropolitan area level.

In [15]:
# pivot = pd.pivot_table(aqs, values="arithmetic_mean", index=["state", "county", "year"], columns=["parameter"])
# 11707 rows × 8 columns

In [16]:
aqs = pd.pivot_table(aqs, values="arithmetic_mean", 
                       index=["state", "county", "year"], 
                       columns=["parameter"],
                       aggfunc='max')
aqs.shape

(12310, 8)

In [17]:
# drop columns with low counts of pollutants or identified by EPA as not
# representative

aqs.drop(columns=['Lead (TSP) LC', 'Lead PM10 LC FRM/FEM', 'Sulfur dioxide'], inplace=True)

In [18]:
# drop rows that don't have all pollutant data and reset index for merging later
aqs.dropna(inplace=True)
aqs.reset_index(inplace=True)

In [19]:
aqs

parameter,state,county,year,Carbon monoxide,Nitrogen dioxide (NO2),Ozone,PM10 Total 0-10um STP,PM2.5 - Local Conditions
0,Alabama,Jefferson,2014,0.578752,28.600838,0.048302,24.918033,11.698347
1,Alabama,Jefferson,2015,0.526681,22.163818,0.047164,23.883333,11.772881
2,Alabama,Jefferson,2016,0.444555,28.544643,0.051325,24.256809,10.781667
3,Alabama,Jefferson,2017,0.367024,20.712610,0.045971,22.300057,10.325455
4,Alabama,Jefferson,2018,0.344179,23.718644,0.049131,22.985533,10.329508
...,...,...,...,...,...,...,...,...
1234,Wyoming,Laramie,2014,0.050685,3.602478,0.055355,10.966667,3.920047
1235,Wyoming,Laramie,2015,0.070951,13.231124,0.052610,12.310345,4.857143
1236,Wyoming,Laramie,2016,0.102225,12.116338,0.052005,15.505759,4.531964
1237,Wyoming,Sweetwater,2011,0.408231,14.536723,0.054370,32.694915,5.110619


In [20]:
# pivot

In [21]:
# pivot = pd.pivot_table(aqs, values="arithmetic_mean", 
#                        index=["state", "county", "year"], 
#                        columns=["parameter"],
#                        aggfunc='max')
# pivot.shape

In [22]:
# pivot.drop(columns=['Lead (TSP) LC', 'Lead PM10 LC FRM/FEM', 'Sulfur dioxide'], inplace=True)

In [23]:
# pivot.dropna(inplace=True) # this model for Winston

In [24]:
# pivot.reset_index(inplace=True)

In [25]:
# pivot

In [26]:
# aqs['parameter'].value_counts()

In [27]:
aqs.shape

(1239, 8)

## EPA Annual Air Quality Index (AQI) Summary Data

### Import Data

The air quality data was downloaded from https://aqs.epa.gov/aqsweb/airdata/download_files.html#Annual as files containing annual data from years 2006-2021.  We are combining these CSV files into a single dataframe for use in this project.  This is summary data that may also be useful in modeling and will be used in parallel with the AQS data above that was collected via API.

In [28]:
# code from Winston merged with approach from
# https://towardsdatascience.com/pandas-concat-tricks-you-should-know-to-speed-up-your-data-analysis-cd3d4fdfe6dd

data_path = Path('data/annual_summary/')

# create a list of all the AQI csv files
aqi_dfs = (
    pd.read_csv(file) for file in data_path.glob('annual_aqi_*') if file.is_file()
)

# concatenate the dataframes
aqi_df = pd.concat(aqi_dfs)

# export the final csv
aqi_df.to_csv('data/annual_summary/annual_aqi_by_county_all.csv', index=False)

Read the final CSV for all the AQI data

In [29]:
aqi = pd.read_csv('data/annual_summary/annual_aqi_by_county_all.csv')

### General Look and Cleanup

In [30]:
aqi.head()

,State,County,Year,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days PM2.5,Days PM10
0,Alabama,Baldwin,2009,252,218,32,2,0,0,0,136,53,36,0,0,200,52,0
1,Alabama,Clay,2009,119,97,22,0,0,0,0,94,59,33,0,0,0,119,0
2,Alabama,Colbert,2009,323,220,103,0,0,0,0,76,60,43,0,0,132,191,0
3,Alabama,DeKalb,2009,363,311,52,0,0,0,0,100,54,36,0,0,308,55,0
4,Alabama,Elmore,2009,244,228,16,0,0,0,0,80,49,36,0,0,244,0,0


In [31]:
aqi.columns

Index(['State', 'County', 'Year', 'Days with AQI', 'Good Days',
       'Moderate Days', 'Unhealthy for Sensitive Groups Days',
       'Unhealthy Days', 'Very Unhealthy Days', 'Hazardous Days', 'Max AQI',
       '90th Percentile AQI', 'Median AQI', 'Days CO', 'Days NO2',
       'Days Ozone', 'Days PM2.5', 'Days PM10'],
      dtype='object')

In [32]:
aqi.shape

(50253, 18)

In [33]:
aqi.dtypes

State                                  object
County                                 object
Year                                    int64
Days with AQI                           int64
Good Days                               int64
Moderate Days                           int64
Unhealthy for Sensitive Groups Days     int64
Unhealthy Days                          int64
Very Unhealthy Days                     int64
Hazardous Days                          int64
Max AQI                                 int64
90th Percentile AQI                     int64
Median AQI                              int64
Days CO                                 int64
Days NO2                                int64
Days Ozone                              int64
Days PM2.5                              int64
Days PM10                               int64
dtype: object

In [34]:
aqi.columns = [col.lower().replace(' ', '_') for col in aqi.columns]

In [35]:
aqi.columns

Index(['state', 'county', 'year', 'days_with_aqi', 'good_days',
       'moderate_days', 'unhealthy_for_sensitive_groups_days',
       'unhealthy_days', 'very_unhealthy_days', 'hazardous_days', 'max_aqi',
       '90th_percentile_aqi', 'median_aqi', 'days_co', 'days_no2',
       'days_ozone', 'days_pm2.5', 'days_pm10'],
      dtype='object')

### Rename Columns
Mainly to trim the length of some of the column names to make it easier to work with.

In [36]:
aqi.rename(columns={
    'moderate_days': 'mod_days',
    'unhealthy_for_sensitive_groups_days': 'uh_sens_days',
    'unhealthy_days': 'uh_days',
    'very_unhealthy_days': 'v_uh_days',
    'hazardous_days': 'haz_days',
    '90th_percentile_aqi': '90_per_aqi'
    }, inplace=True)

In [37]:
aqi.isnull().sum()[aqi.isnull().sum() > 0]

Series([], dtype: int64)

### Check County and State Information

Mainly looking for outliers or any text in the names that might be cause for further investigation, especially since we will be merging this data with the CDC data based on year, state, and county.

In [38]:
aqi[['state', 'county']].nunique()

state      55
county    913
dtype: int64

In [39]:
aqi['state'].unique()

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Country Of Mexico', 'Delaware',
       'District Of Columbia', 'Florida', 'Georgia', 'Hawaii', 'Idaho',
       'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana',
       'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota',
       'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada',
       'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
       'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon',
       'Pennsylvania', 'Puerto Rico', 'Rhode Island', 'South Carolina',
       'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont',
       'Virgin Islands', 'Virginia', 'Washington', 'West Virginia',
       'Wisconsin', 'Wyoming', 'Canada'], dtype=object)

May want to drop locations such as:
* Country of Mexico
* Puerto Rico
* Virgin Islands
* Canada

In [40]:
aqi.drop(
    aqi[
        aqi['state'].isin(
            ['Country Of Mexico', 'Puerto Rico', 'Virgin Islands', 'Canada']
        )
    ].index, inplace=True)

In [41]:
# commented out to save space in the published notebook
# aqi['county'].unique()

In [42]:
aqi.head(3)

,state,county,year,days_with_aqi,good_days,mod_days,uh_sens_days,uh_days,v_uh_days,haz_days,max_aqi,90_per_aqi,median_aqi,days_co,days_no2,days_ozone,days_pm2.5,days_pm10
0,Alabama,Baldwin,2009,252,218,32,2,0,0,0,136,53,36,0,0,200,52,0
1,Alabama,Clay,2009,119,97,22,0,0,0,0,94,59,33,0,0,0,119,0
2,Alabama,Colbert,2009,323,220,103,0,0,0,0,76,60,43,0,0,132,191,0


In [43]:
aqi.describe()

,year,days_with_aqi,good_days,mod_days,uh_sens_days,uh_days,v_uh_days,haz_days,max_aqi,90_per_aqi,median_aqi,days_co,days_no2,days_ozone,days_pm2.5,days_pm10
count,49503.000000,49503.000000,49503.000000,49503.000000,49503.000000,49503.000000,49503.000000,49503.000000,49503.00000,49503.000000,49503.000000,49503.000000,49503.000000,49503.000000,49503.000000,49503.000000
mean,2013.383553,306.444882,234.848615,65.491546,5.049815,0.913581,0.094297,0.047027,127.41143,61.653718,37.645961,0.981395,6.662748,171.259802,113.697594,13.843343
std,4.605960,89.579243,82.746696,51.766251,10.383755,4.010577,1.102682,0.678465,223.14044,19.369735,10.936162,12.173768,26.480322,119.197510,110.489383,52.572736
min,2006.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.00000,5.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2009.000000,256.000000,177.000000,26.000000,0.000000,0.000000,0.000000,0.000000,90.00000,50.000000,33.000000,0.000000,0.000000,59.000000,0.000000,0.000000
50%,2013.000000,360.000000,250.000000,53.000000,1.000000,0.000000,0.000000,0.000000,112.00000,59.000000,39.000000,0.000000,0.000000,187.000000,95.000000,0.000000
75%,2017.000000,365.000000,304.000000,94.000000,6.000000,0.000000,0.000000,0.000000,146.00000,71.000000,44.000000,0.000000,0.000000,245.000000,185.000000,1.000000
max,2021.000000,366.000000,365.000000,339.000000,122.000000,92.000000,74.000000,37.000000,14043.00000,306.000000,132.000000,365.000000,365.000000,366.000000,366.000000,366.000000


All the values seem like they would be in range--meaning no negative counts, no days over 366.

## CDC Data

Want to import the CDC data and compare the counties.

We are using the WONDER tool from the CDC to export the files that we are importing to this notebook. For both files, we filtered out any births with maternal risk factors so we they are not influencing birth weight.  There are some years that have no or suppressed data, which are omitted from the CDC export and are not present.We filtered out any births with maternal risk factors so we they are not influencing birth weight.  There are some years that have no or suppressed data, which are omitted from the CDC export and are not present. 

There are two main files:
1. `Natality_low_2007-2021` which represents all the births below 2500 grams by year and county. 
1. `Natality_all_2007-2021` which represents all the births by year and county.

### Import Data

In [44]:
cdc_low = pd.read_csv('data/Natality_low_2007-2021.txt', sep='\t')
cdc_all = pd.read_csv('data/Natality_all_2007-2021.txt', sep='\t')

### General Look and Cleanup

In [45]:
cdc_low.head()

,Notes,State,State Code,County,County Code,Year,Year Code,Births,Average Birth Weight,Average LMP Gestational Age,Average OE Gestational Age
0,NaN,Alabama,1.0,"Baldwin County, AL",1003.0,2014.0,2014.0,137.0,1995.255,34.241,34.051
1,NaN,Alabama,1.0,"Baldwin County, AL",1003.0,2015.0,2015.0,141.0,1832.837,32.965,32.922
2,NaN,Alabama,1.0,"Baldwin County, AL",1003.0,2016.0,2016.0,120.0,2008.358,34.467,34.058
3,NaN,Alabama,1.0,"Baldwin County, AL",1003.0,2017.0,2017.0,100.0,2082.830,34.550,34.380
4,NaN,Alabama,1.0,"Baldwin County, AL",1003.0,2018.0,2018.0,122.0,1942.303,33.762,33.508


In [46]:
cdc_all.head()

,Notes,State,State Code,County,County Code,Year,Year Code,Births,Average Birth Weight,Average LMP Gestational Age,Average OE Gestational Age
0,NaN,Alabama,1.0,"Baldwin County, AL",1003.0,2014.0,2014.0,1724.0,3296.472,38.595,38.489
1,NaN,Alabama,1.0,"Baldwin County, AL",1003.0,2015.0,2015.0,1857.0,3290.811,38.487,38.347
2,NaN,Alabama,1.0,"Baldwin County, AL",1003.0,2016.0,2016.0,1722.0,3315.387,38.690,38.497
3,NaN,Alabama,1.0,"Baldwin County, AL",1003.0,2017.0,2017.0,1782.0,3346.588,38.713,38.572
4,NaN,Alabama,1.0,"Baldwin County, AL",1003.0,2018.0,2018.0,1770.0,3301.324,38.580,38.423


In [47]:
cdc_low.dtypes

Notes                           object
State                           object
State Code                     float64
County                          object
County Code                    float64
Year                           float64
Year Code                      float64
Births                         float64
Average Birth Weight           float64
Average LMP Gestational Age    float64
Average OE Gestational Age     float64
dtype: object

In [48]:
cdc_all.dtypes

Notes                           object
State                           object
State Code                     float64
County                          object
County Code                    float64
Year                           float64
Year Code                      float64
Births                         float64
Average Birth Weight           float64
Average LMP Gestational Age    float64
Average OE Gestational Age     float64
dtype: object

In [49]:
cdc_low.shape

(6712, 11)

In [50]:
cdc_all.shape

(6715, 11)

In [51]:
cdc_low.columns, cdc_all.columns

(Index(['Notes', 'State', 'State Code', 'County', 'County Code', 'Year',
        'Year Code', 'Births', 'Average Birth Weight',
        'Average LMP Gestational Age', 'Average OE Gestational Age'],
       dtype='object'),
 Index(['Notes', 'State', 'State Code', 'County', 'County Code', 'Year',
        'Year Code', 'Births', 'Average Birth Weight',
        'Average LMP Gestational Age', 'Average OE Gestational Age'],
       dtype='object'))

In [52]:
cdc_low.columns = [col.lower().replace(' ', '_') for col in cdc_low.columns]
cdc_all.columns = [col.lower().replace(' ', '_') for col in cdc_all.columns]

Renaming the % of births column, and if we want different names for other columns, we can do it at this step

In [53]:
cdc_low.columns

Index(['notes', 'state', 'state_code', 'county', 'county_code', 'year',
       'year_code', 'births', 'average_birth_weight',
       'average_lmp_gestational_age', 'average_oe_gestational_age'],
      dtype='object')

In [54]:
cdc_low['notes'].nunique(), cdc_all['notes'].nunique()

(91, 87)

In [55]:
# cdc_all['notes'].unique()

All the notes look like they are what shows up at the bottom of the file and isn't data that we're looking for.  Dropping the notes column

In [56]:
cdc_low.drop(['notes'], axis=1, inplace=True)
cdc_all.drop(['notes'], axis=1, inplace=True)

In [57]:
cdc_low.isna().sum()[cdc_low.isna().sum() > 0]

state                          96
state_code                     96
county                         96
county_code                    96
year                           96
year_code                      96
births                         96
average_birth_weight           96
average_lmp_gestational_age    96
average_oe_gestational_age     96
dtype: int64

In [58]:
cdc_all.isna().sum()[cdc_all.isna().sum() > 0]

state                          92
state_code                     92
county                         92
county_code                    92
year                           92
year_code                      92
births                         92
average_birth_weight           92
average_lmp_gestational_age    92
average_oe_gestational_age     92
dtype: int64

In [59]:
cdc_low.tail(3)

,state,state_code,county,county_code,year,year_code,births,average_birth_weight,average_lmp_gestational_age,average_oe_gestational_age
6709,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6710,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6711,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
cdc_all.tail(3)

,state,state_code,county,county_code,year,year_code,births,average_birth_weight,average_lmp_gestational_age,average_oe_gestational_age
6712,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6713,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We need state and county information for this project,and it looks like all these null values belong, possibly to rows that were holding that notes data.  Dropping these rows to see if that clears up all the nulls.

Drop the rows that have the footnotes in them

In [61]:
cdc_low.dropna(subset=['state'], inplace=True)
cdc_all.dropna(subset=['state'], inplace=True)

In [62]:
cdc_low.isna().sum()[cdc_low.isna().sum() > 0]

Series([], dtype: int64)

In [63]:
cdc_all.isna().sum()[cdc_all.isna().sum() > 0]

Series([], dtype: int64)

That did resolve all the null values.

In [64]:
cdc_low.shape, cdc_all.shape

((6616, 10), (6623, 10))

In [65]:
cdc_low.tail(3)

,state,state_code,county,county_code,year,year_code,births,average_birth_weight,average_lmp_gestational_age,average_oe_gestational_age
6613,Wyoming,56.0,"Unidentified Counties, WY",56999.0,2019.0,2019.0,335.0,2062.824,35.051,34.919
6614,Wyoming,56.0,"Unidentified Counties, WY",56999.0,2020.0,2020.0,307.0,2088.586,35.365,34.987
6615,Wyoming,56.0,"Unidentified Counties, WY",56999.0,2021.0,2021.0,319.0,2063.197,34.821,34.862


In [66]:
cdc_all.tail(3)

,state,state_code,county,county_code,year,year_code,births,average_birth_weight,average_lmp_gestational_age,average_oe_gestational_age
6620,Wyoming,56.0,"Unidentified Counties, WY",56999.0,2019.0,2019.0,4592.0,3218.040,38.709,38.636
6621,Wyoming,56.0,"Unidentified Counties, WY",56999.0,2020.0,2020.0,4285.0,3228.561,38.733,38.641
6622,Wyoming,56.0,"Unidentified Counties, WY",56999.0,2021.0,2021.0,4379.0,3227.242,38.608,38.551


It looks like year and year_code might contain the same data

In [67]:
(cdc_low['year'] == cdc_low['year_code']).sum() == cdc_low.shape[0]

True

`year` and `year_code` appear to be the same.  We also don't have state or county codes in the EPA data, so dropping those as well.

In [68]:
cdc_low.drop(['year_code', 'county_code', 'state_code'], axis=1, inplace=True)
cdc_all.drop(['year_code', 'county_code', 'state_code'], axis=1, inplace=True)

Also want the year as an int not a float

In [69]:
cdc_low['year'] = cdc_low['year'].astype(int)
cdc_all['year'] = cdc_all['year'].astype(int)

In [70]:
cdc_low.columns

Index(['state', 'county', 'year', 'births', 'average_birth_weight',
       'average_lmp_gestational_age', 'average_oe_gestational_age'],
      dtype='object')

### Gestational Age at Birth
Beginning in 2014 NCHS changed the standard for gestational period from the Last Menstrual Period (LMP) based gestational age to the Obstetric/clinical Estimate (OE) based gestational age. Obstetric/clinical Estimate (OE) based gestational age groups are available for years 2007 and later in WONDER since February 2016. Refer to [Measuring Gestational Age in Vital Statistics Data: Transitioning to the Obstetric Estimate](http://www.cdc.gov/nchs/data/nvsr/nvsr64/nvsr64_05.pdf) for more information.

I believe this means this means we should drop the LMP column and keep the OE column.

In [71]:
cdc_low.drop(['average_lmp_gestational_age'], axis=1, inplace=True)
cdc_all.drop(['average_lmp_gestational_age'], axis=1, inplace=True)

In [72]:
cdc_low.columns

Index(['state', 'county', 'year', 'births', 'average_birth_weight',
       'average_oe_gestational_age'],
      dtype='object')

### Rename Columns

To prepare for merging this data, giving the columns specific names to indicate which dataframe they came from.

In [73]:
cdc_low.rename(columns={
    'births': 'births_low',
    'average_birth_weight': 'avg_weight_low',
    'average_oe_gestational_age': 'avg_ges_age_low'
    }, inplace=True)

cdc_all.rename(columns={
    'births': 'births_all',
    'average_birth_weight': 'avg_weight_all',
    'average_oe_gestational_age': 'avg_ges_age_all'
    }, inplace=True)

In [74]:
cdc_low.head(3)

,state,county,year,births_low,avg_weight_low,avg_ges_age_low
0,Alabama,"Baldwin County, AL",2014,137.0,1995.255,34.051
1,Alabama,"Baldwin County, AL",2015,141.0,1832.837,32.922
2,Alabama,"Baldwin County, AL",2016,120.0,2008.358,34.058


In [75]:
cdc_all.head(3)

,state,county,year,births_all,avg_weight_all,avg_ges_age_all
0,Alabama,"Baldwin County, AL",2014,1724.0,3296.472,38.489
1,Alabama,"Baldwin County, AL",2015,1857.0,3290.811,38.347
2,Alabama,"Baldwin County, AL",2016,1722.0,3315.387,38.497


### Merge CDC Data

Before further cleaning, the dataframes will be merged.  The low birthweight dataframe `cdc_low` is the main data and we will supplement that with the all birthweight data `cdc_all` so we should end up with a single dataframe that contains 1 entry for each year/county combination that has both the low birthweight and all birthweight data.

In [76]:
cdc = pd.merge(cdc_low, cdc_all, how='left', on=['state', 'county', 'year'])

In [77]:
cdc.head()

,state,county,year,births_low,avg_weight_low,avg_ges_age_low,births_all,avg_weight_all,avg_ges_age_all
0,Alabama,"Baldwin County, AL",2014,137.0,1995.255,34.051,1724.0,3296.472,38.489
1,Alabama,"Baldwin County, AL",2015,141.0,1832.837,32.922,1857.0,3290.811,38.347
2,Alabama,"Baldwin County, AL",2016,120.0,2008.358,34.058,1722.0,3315.387,38.497
3,Alabama,"Baldwin County, AL",2017,100.0,2082.830,34.380,1782.0,3346.588,38.572
4,Alabama,"Baldwin County, AL",2018,122.0,1942.303,33.508,1770.0,3301.324,38.423


### County Names

In [78]:
cdc['county'].sample(10)

4559          Allen County, OH
6354      Snohomish County, WA
3109        Oakland County, MI
3852         Broome County, NY
2286        Johnson County, IA
1421       Sarasota County, FL
602     San Joaquin County, CA
5989           Webb County, TX
3068           Kent County, MI
5229         Butler County, PA
Name: county, dtype: object

I ran the sample code above multiple times to see some of the data in the county column.  Looks like they are including the state abbreviation in the County name, so will remove that.  There were also some places that didn't say "County" and said "Counties" (i.e. Unidentified Counties, WY) and also there were some locations that were parishes (in LA) or other unique monikers like Borough.  So I'm splitting on both the " Count" to capture County and Counties as well as "," to get both cases.

We should look at what the AQI data has and decide whether we need to treat the remaining unique naming cases that remain after this clean up.

In [79]:
cdc['county'] = cdc['county'
                   ].apply(lambda x: x.split(' Count')[0].split(',')[0].title())

In [80]:
cdc[['state', 'county']].nunique()

state      50
county    466
dtype: int64

In [81]:
cdc['county'].sample(10)

5273              Fayette
3108              Oakland
2132              Madison
2400             Sedgwick
3903                 Erie
6246    Newport News City
6566           Washington
3845                Bronx
6594         Unidentified
6472            La Crosse
Name: county, dtype: object

This looks more like what I would expect where it is just the name of the county.  Since we also saw that there were unidentified counties, and they are all just listed as Unidentified now, we should drop that data since we are trying to investigate county air quality and county birth weights, so without a specific county, the data is not valuable to us.

In [82]:
# looking through all the unique values for anything else interesting
# commented out to save space in the published notebook
# cdc['county'].unique()

I'm also seeing many counties that end in lower case 'city' so investigating some of those, like:
* St. Louis city
* Chesapeake city
* Norfolk city
* Portsmouth city

St. Louis is interesting, since it sounds like the city is a county, but there is also a St. Louis county that surrounds the city according to https://www.stlouis-mo.gov/government/about/city-government-structure.cfm

Chesapeake, Virginia is also an independent city and not part of a county https://en.wikipedia.org/wiki/Chesapeake,_Virginia

Given this information and the information above about seeing some "parish" listings in Louisiana, we checked against the AQI county listings and found that in the AQI data:
* the word Parish does not appear, but the name of the Parish does--like 'East Baton Rouge', so we should drop 'Parish' from the CDC county names.
* The word City does appear in the AQI data, but it is capitalized, so we should title case the CDC data or make all of them lower case when we go to match them up (this has been incorporated into earlier cleaning steps and is no longer a concern)

In [83]:
# drop parish
cdc['county'] = cdc['county'
                   ].apply(lambda x: x.split(' Parish')[0])

In [84]:
cdc.shape

(6616, 9)

In [85]:
cdc.drop(cdc[cdc['county'] == 'Unidentified'].index, inplace=True)

In [86]:
cdc.shape

(6094, 9)

In [87]:
cdc.dtypes

state               object
county              object
year                 int64
births_low         float64
avg_weight_low     float64
avg_ges_age_low    float64
births_all         float64
avg_weight_all     float64
avg_ges_age_all    float64
dtype: object

### Calculate Percent of Low Birthweight

We have the total births and the low birthweight births, so let's create a percentage of low birthrate births before exporting the cleaned data.

The current national average is 8.24% (.0824), so we'll create a boolean feature as well to indicate if the observation is above the national average

In [88]:
cdc['pct_low'] = (cdc['births_low'] / cdc['births_all']).round(3)

In [89]:
cdc['high_rate'] = cdc['pct_low'].apply(lambda x: 1 if x > .0824 else 0)

In [90]:
cdc.head(10)

,state,county,year,births_low,avg_weight_low,avg_ges_age_low,births_all,avg_weight_all,avg_ges_age_all,pct_low,high_rate
0,Alabama,Baldwin,2014,137.0,1995.255,34.051,1724.0,3296.472,38.489,0.079,0
1,Alabama,Baldwin,2015,141.0,1832.837,32.922,1857.0,3290.811,38.347,0.076,0
2,Alabama,Baldwin,2016,120.0,2008.358,34.058,1722.0,3315.387,38.497,0.070,0
3,Alabama,Baldwin,2017,100.0,2082.830,34.380,1782.0,3346.588,38.572,0.056,0
4,Alabama,Baldwin,2018,122.0,1942.303,33.508,1770.0,3301.324,38.423,0.069,0
5,Alabama,Baldwin,2019,99.0,2060.768,34.919,1845.0,3327.293,38.521,0.054,0
6,Alabama,Baldwin,2020,113.0,2025.478,34.982,1763.0,3305.012,38.529,0.064,0
7,Alabama,Baldwin,2021,120.0,1969.508,34.269,1969.0,3293.984,38.394,0.061,0
8,Alabama,Calhoun,2014,59.0,1972.661,34.475,948.0,3275.708,38.668,0.062,0
9,Alabama,Calhoun,2015,65.0,1909.708,33.892,983.0,3267.121,38.590,0.066,0


In [91]:
cdc['high_rate'].value_counts(normalize=True)

0    0.910896
1    0.089104
Name: high_rate, dtype: float64

## Merge CDC and EPA Data

Since we've found that the AQS data has more detail, but we have fewer counties that have a larger set of pollutant data, we will create both a set of merged data from the AQS API data and a set from the AQI Annual Summary data.

Both will be merged in a way where the current year of air quality data (AQS or AQI) will be merged with the following year CDC data.  The assumption here is that the impacts of poor air quality during pregnancy will appear in the following year's birth weight data and then any models that we build to classify or regress this data will have some predictive power for the following year based on the current year's air quality.

In [92]:
# county_aqs.head(3)

### Create year_for_merge column in the CDC data

In [93]:
cdc['year_for_merge'] = cdc['year'] -1

In [94]:
cdc.head(3)

,state,county,year,births_low,avg_weight_low,avg_ges_age_low,births_all,avg_weight_all,avg_ges_age_all,pct_low,high_rate,year_for_merge
0,Alabama,Baldwin,2014,137.0,1995.255,34.051,1724.0,3296.472,38.489,0.079,0,2013
1,Alabama,Baldwin,2015,141.0,1832.837,32.922,1857.0,3290.811,38.347,0.076,0,2014
2,Alabama,Baldwin,2016,120.0,2008.358,34.058,1722.0,3315.387,38.497,0.070,0,2015


### Merge AQS and CDC Data

In [95]:
aqs_merged = pd.merge(aqs, cdc, how='inner', 
                  left_on=['state', 'county', 'year'],
                  right_on=['state', 'county', 'year_for_merge'])

In [96]:
aqs_merged.head(3)

,state,county,year_x,Carbon monoxide,Nitrogen dioxide (NO2),Ozone,PM10 Total 0-10um STP,PM2.5 - Local Conditions,year_y,births_low,avg_weight_low,avg_ges_age_low,births_all,avg_weight_all,avg_ges_age_all,pct_low,high_rate,year_for_merge
0,Alabama,Jefferson,2014,0.578752,28.600838,0.048302,24.918033,11.698347,2015,687.0,1940.866,34.068,7376.0,3216.859,38.508,0.093,1,2014
1,Alabama,Jefferson,2015,0.526681,22.163818,0.047164,23.883333,11.772881,2016,681.0,1903.761,33.633,7032.0,3210.182,38.434,0.097,1,2015
2,Alabama,Jefferson,2016,0.444555,28.544643,0.051325,24.256809,10.781667,2017,709.0,1947.176,34.059,7061.0,3188.066,38.456,0.100,1,2016


In [97]:
aqs_merged.shape

(837, 18)

In [98]:
aqs_merged.columns

Index(['state', 'county', 'year_x', 'Carbon monoxide',
       'Nitrogen dioxide (NO2)', 'Ozone', 'PM10 Total 0-10um STP',
       'PM2.5 - Local Conditions', 'year_y', 'births_low', 'avg_weight_low',
       'avg_ges_age_low', 'births_all', 'avg_weight_all', 'avg_ges_age_all',
       'pct_low', 'high_rate', 'year_for_merge'],
      dtype='object')

In [99]:
aqs_merged.drop(['year_y', 'year_for_merge'],
            axis=1, 
            inplace=True)

aqs_merged.rename(columns={'year_x': 'year'}, inplace=True)

In [100]:
aqs_merged.head(3)

,state,county,year,Carbon monoxide,Nitrogen dioxide (NO2),Ozone,PM10 Total 0-10um STP,PM2.5 - Local Conditions,births_low,avg_weight_low,avg_ges_age_low,births_all,avg_weight_all,avg_ges_age_all,pct_low,high_rate
0,Alabama,Jefferson,2014,0.578752,28.600838,0.048302,24.918033,11.698347,687.0,1940.866,34.068,7376.0,3216.859,38.508,0.093,1
1,Alabama,Jefferson,2015,0.526681,22.163818,0.047164,23.883333,11.772881,681.0,1903.761,33.633,7032.0,3210.182,38.434,0.097,1
2,Alabama,Jefferson,2016,0.444555,28.544643,0.051325,24.256809,10.781667,709.0,1947.176,34.059,7061.0,3188.066,38.456,0.100,1


### Merge AQI and CDC Data

In [101]:
aqi_merged = pd.merge(aqi, cdc, how='inner', 
                  left_on=['state', 'county', 'year'],
                  right_on=['state', 'county', 'year_for_merge'])

In [102]:
aqi_merged.head(3)

,state,county,year_x,days_with_aqi,good_days,mod_days,uh_sens_days,uh_days,v_uh_days,haz_days,...,year_y,births_low,avg_weight_low,avg_ges_age_low,births_all,avg_weight_all,avg_ges_age_all,pct_low,high_rate,year_for_merge
0,California,Alameda,2009,365,190,162,12,1,0,0,...,2010,1136.0,1980.992,34.26,17700.0,3314.9,38.817,0.064,0,2009
1,California,Alameda,2009,365,190,162,12,1,0,0,...,2010,1136.0,1980.992,34.26,17700.0,3314.9,38.817,0.064,0,2009
2,California,Alameda,2009,365,190,162,12,1,0,0,...,2010,1136.0,1980.992,34.26,17700.0,3314.9,38.817,0.064,0,2009


In [103]:
aqi_merged.shape

(15441, 28)

In [104]:
aqi_merged.columns

Index(['state', 'county', 'year_x', 'days_with_aqi', 'good_days', 'mod_days',
       'uh_sens_days', 'uh_days', 'v_uh_days', 'haz_days', 'max_aqi',
       '90_per_aqi', 'median_aqi', 'days_co', 'days_no2', 'days_ozone',
       'days_pm2.5', 'days_pm10', 'year_y', 'births_low', 'avg_weight_low',
       'avg_ges_age_low', 'births_all', 'avg_weight_all', 'avg_ges_age_all',
       'pct_low', 'high_rate', 'year_for_merge'],
      dtype='object')

In [105]:
aqi_merged.drop(['year_y', 'year_for_merge'],
            axis=1, 
            inplace=True)

aqi_merged.rename(columns={'year_x': 'year'}, inplace=True)

In [106]:
aqi_merged.head(3)

,state,county,year,days_with_aqi,good_days,mod_days,uh_sens_days,uh_days,v_uh_days,haz_days,...,days_pm2.5,days_pm10,births_low,avg_weight_low,avg_ges_age_low,births_all,avg_weight_all,avg_ges_age_all,pct_low,high_rate
0,California,Alameda,2009,365,190,162,12,1,0,0,...,234,0,1136.0,1980.992,34.26,17700.0,3314.9,38.817,0.064,0
1,California,Alameda,2009,365,190,162,12,1,0,0,...,234,0,1136.0,1980.992,34.26,17700.0,3314.9,38.817,0.064,0
2,California,Alameda,2009,365,190,162,12,1,0,0,...,234,0,1136.0,1980.992,34.26,17700.0,3314.9,38.817,0.064,0


## Export Cleaned Data

In [107]:
# cdc.to_csv('data/cdc_cleaned.csv', index=False)
aqs_merged.to_csv('data/aqs_by_county_clean.csv', index=False)
aqi_merged.to_csv('data/annual_aqi_clean.csv', index=False)

# PARKING LOT

In [108]:
# merged[['state', 'county']].nunique()

In [109]:
# merged['high_rate'].value_counts(normalize=True)

In [110]:
# merged['state'].value_counts()

In [111]:
# pivot.drop(columns=['PM10 Total 0-10um STP']).dropna()

In [113]:
# all_params = pivot.dropna().reset_index()

In [ ]:
# all_params

Pivot table
Get index from pivot table
re-index original 
get primary_exceedance_count from this

In [ ]:
# pivot.columns

In [ ]:
# pivot[['PM2.5 - Local Conditions', 'Ozone']].dropna()

Unstack approach

In [ ]:
# county_aqs = aqs.groupby(['state', 'county', 'year', 'parameter'])['arithmetic_mean'].mean().unstack().reset_index()

In [ ]:
# county_aqs.head(20)

In [ ]:
# county_aqs[['Ozone', 'Sulfur dioxide']].dropna()

Looking at example data that Winston collected via API

In [ ]:
# df.drop_duplicates()

In [ ]:
# df = pd.read_csv('data/AQS_data_test_Alabama_2006.csv')

In [ ]:
# df.head()

In [ ]:
# df.columns

In [ ]:
# df['sample_duration_code'].unique()

In [ ]:
# df[['parameter', 'sample_duration_code']].value_counts()

In [ ]:
# df.drop(['Unnamed: 0', 'state_code', 'county_code'], axis=1, inplace=True)

In [ ]:
# df.shape

In [ ]:
# df['county'].unique()

In [ ]:
# df[(df['state'] == 'Alabama') & (df['county'] == 'Shelby')]['pollutant_standard'].unique()

Since the API call failed me, I found that the annual concentration data looked fairly similar to what we were collecting, so giving that a shot.  I downloaded the annual files for years 2006-2021, so we'll take a shot at seeing if this data can be worked into a solution

In [ ]:
# # code from Winston merged with approach from
# # https://towardsdatascience.com/pandas-concat-tricks-you-should-know-to-speed-up-your-data-analysis-cd3d4fdfe6dd

# data_path = Path('data/')

# # create a list of all the AQI csv files
# dfs = (
#     pd.read_csv(file, low_memory=False) for file in data_path.glob('*_conc_*') if file.is_file()
# )

# # concatenate the dataframes
# df = pd.concat(dfs)

# # export the final csv
# # res.to_csv('data/aqi_by_year_2006-2021.csv', index=False)

In [ ]:
# # in the current data, I think this is akin to completeness_indicator
# df = df[df['completeness_indicator']=='Y']

In [ ]:
# df['Year'].value_counts()

In [ ]:
# df.columns

In [ ]:
# df.columns = [col.lower().replace(' ', '_') for col in df.columns]

In [ ]:
# df.columns

In [ ]:
# df.isna().sum()[df.isna().sum() > 0]

In [ ]:
# df['completeness_indicator'].value_counts()

In [ ]:
# df.head()

In [ ]:
# df.shape

In [ ]:
# births = pd.read_csv('data/cdc_cleaned.csv')

In [ ]:
# births.head()

In [ ]:
# births.shape

In [ ]:
# births.describe()

In [ ]:
# births['year'].value_counts()

In [ ]:
# set(births[births['year'] == 2007]['county'])

In [ ]:
# how many counties from the aqi are also in the cdc data
# len(set(aqi['county']).intersection(set(cdc['County'])))

In [ ]:
# what are the differences
#set(aqi['county']).difference(set(cdc['County']))

### EPA API Information

Look to see if getting more granular data from the API is feasible

Your user ID is your email address: mrhurless@gmail.com 
Your key is: rubymouse94 

In [ ]:
# import requests

In [ ]:
# email = 'mrhurless@gmail.com'
# key = 'rubymouse94'

# url = f'https://aqs.epa.gov/data/api/list/parametersByClass?email={email}&key={key}&pc=CRITERIA'

# res = requests.get(url)

# res.status_code

In [ ]:
# # from https://stackoverflow.com/questions/71603314/ssl-error-unsafe-legacy-renegotiation-disabled

# import urllib3
# import ssl

# class CustomHttpAdapter (requests.adapters.HTTPAdapter):
#     # "Transport adapter" that allows us to use custom ssl_context.

#     def __init__(self, ssl_context=None, **kwargs):
#         self.ssl_context = ssl_context
#         super().__init__(**kwargs)

#     def init_poolmanager(self, connections, maxsize, block=False):
#         self.poolmanager = urllib3.poolmanager.PoolManager(
#             num_pools=connections, maxsize=maxsize,
#             block=block, ssl_context=self.ssl_context)


# def get_legacy_session():
#     ctx = ssl.create_default_context(ssl.Purpose.SERVER_AUTH)
#     ctx.options |= 0x4  # OP_LEGACY_SERVER_CONNECT
#     session = requests.session()
#     session.mount('https://', CustomHttpAdapter(ctx))
#     return session

In [ ]:
# res = get_legacy_session().get(url)

In [ ]:
# def get_aqi():
#     creds = ('mhurless@me.com', '0n3L0v3#')
#     email = 'mrhurless@gmail.com'
#     key = 'rubymouse94'
#     all_posts = []
    
#     url = "https://support.brightsign.biz/api/v2/community/posts"
    
#     res = requests.get(url, auth=creds)
    
#     # get data other than posts data
#     metadata = {
#                 key: value for key, value in res.json().items() 
#                 if key != 'posts'
#             }
    
#     last_page = metadata['page_count']
#     current_page = metadata['page']
    
#     #count = 0 #keep track of posts # don't think this is needed with this
#     # approach

#     # goal is to get posts from all pages, so we'll check if we've reached 
#     # the last page, but will also put in a catch for the rate limit as 
#     # mentioned by the API docs
    
#     while current_page <= last_page:
#         res = requests.get(url, auth=creds)

#         if res.status_code == 200:
#             posts = pd.DataFrame(res.json()['posts'])
#             metadata = {
#                 key: value for key, value in res.json().items() 
#                 if key != 'posts'
#             }
            
#             # update current page
#             current_page = metadata['page']
            
#             # update URL to pull posts from next page
#             url = metadata['next_page']

#             all_posts.append(posts)
            
#             #sleep(5)

#             if metadata['page'] == metadata['page_count']: 
#                 break #break loop if last page is reached

#             #get sequential posts from most recent to least    
#             #params['before'] = posts['created_utc'].min()
#         elif res.status_code == 429:
#             sleep(res.headers['retry-after'])
#         else:
#             print(f'status: {res.status_code}')  
#     print(f'posts retrieved from API: {len(all_posts)}')

#     return pd.concat(all_posts)